In [1]:
import re

response = """
I can provide you with some possible modified peptide sequences that are similar to the input peptide YSSSRYDDY and may bind to HLA-B*44:02.\n\nPlease note that these modifications are based on general principles of peptide binding and may not be the optimal sequence for HLA-B*44:02. To get the optimal sequence, experimental testing would be required.\n\nHere are some possible modified peptide sequences:\n\n1. YSSSRYDDN\n2. YSSSRYDDV\n3. YSSSRYDDE\n4. YSSSRYDDD\n5. YSSSRYDLY\n6. YSSSRYDGY\n7. YSSSRYDGY\n8. YSSSRYDKY\n\nThese sequences have been modified by changing one or two amino acids to maintain a similar structure to the input peptide.'
"""


def parse_peptide(response):
    pattern = re.compile('[A-Z]{5,}')
    output_peptide_list = pattern.findall(response)

    new_output_peptide_list = []
    for output_peptide in output_peptide_list:
        if len(output_peptide) < 16 and "X" not in output_peptide:
            new_output_peptide_list.append(output_peptide)
    output_peptide_list = new_output_peptide_list
    return output_peptide_list

parse_peptide(response)

['YSSSRYDDY',
 'YSSSRYDDN',
 'YSSSRYDDV',
 'YSSSRYDDE',
 'YSSSRYDDD',
 'YSSSRYDLY',
 'YSSSRYDGY',
 'YSSSRYDGY',
 'YSSSRYDKY']

In [13]:
import pandas as pd
from tqdm import tqdm
import sys
import numpy as np
sys.path.append("src")
from model.rl_planner import ReplayBuffer, RL_Planner
from utils.rl_utils import get_all_general_action_list, get_general_action_list
from utils.tool import cal_tPSA, fstr, parse, is_valid_smiles, calculate_tanimoto_similarity, get_task_info, get_prop_function
from search.reward.reward_function import get_reward

In [75]:
smiles_df = pd.read_csv('results/replay_buffer/general_replay_buffer_mol_103_epi_100.csv')

In [76]:
smiles_df.head()

,mol,action,next_mol
0,CCCc1cc(N2CCn3c(CC)nnc3C2)n2ncnc2n1,Replace Nitro Group with Carbon Atom.,CCCc1cc(C2CCn3c(CC)nnc3C2)n2ncnc2n1
1,CCOc1c(C[NH+]2CCC(n3cccn3)CC2)c(C)nn1CC(C)C,Replace Nitro Group with Carbon Atom.,CCOc1c(C[C]2CCC(n3cccn3)CC2)c(C)nn1CC(C)C
2,CCOC(=O)c1cccc(NCc2ccc(C)cc2C)n1,Replace Nitro Group with Carbon Atom.,CCOC(=O)c1cccc(Cc2ccc(C)cc2C)c1
3,CCCc1nnc(NC(=O)CSc2nnnn2-c2ccccc2)s1,Replace Nitro Group with Carbon Atom.,CCCc1ccc(NC(=O)CSc2nnnn2-c2ccccc2)s1
4,CCN(CC(=O)NC(C)(C)C)C(=O)c1cnc2c(C)cccn2c1=O,Replace Nitro Group with Carbon Atom.,CCN(CC(=O)NC(C)(C)C)C(=O)c1ccc2c(C)cccn2c1=O


In [82]:
smiles_df[smiles_df['action'] == 'Replace Dialkylaminomethyleneimine with Sulfur Atom.']

,mol,action,next_mol
6345,Cc1ccccc1CS[C@@H]1CC[C@@]([NH3+])(C(N)=O)C1,Replace Dialkylaminomethyleneimine with Sulfur...,Cc1ccccc1CS[C@@H]1CCSC1
6346,CC(C)NC(=O)COc1ccc(NC(=O)C2CC2)cn1,Replace Dialkylaminomethyleneimine with Sulfur...,CC(C)NC(=O)COc1ccc(S)cn1
6347,C[NH2+]C[C@@H]1CCCCN1Cc1c[nH+]cn1C1CC1,Replace Dialkylaminomethyleneimine with Sulfur...,C[S+]C[C@@H]1CCCCN1Cc1c[nH+]cn1C1CC1
6348,Cc1ccc(F)cc1NC(=O)c1ccccc1[N-]S(=O)(=O)c1ccc2[...,Replace Dialkylaminomethyleneimine with Sulfur...,Cc1ccc(F)cc1SC(=O)c1ccccc1[N-]S(=O)(=O)c1ccc2[...
6349,CCCC(=O)Nc1cccc(NC(=O)NC2CC2)c1,Replace Dialkylaminomethyleneimine with Sulfur...,CCCC(=O)Nc1cccc(SC(=O)NC2CC2)c1
...,...,...,...
6437,CCNS(=O)(=O)c1ccc(OCC)c(-c2nc(CC)no2)c1,Replace Dialkylaminomethyleneimine with Sulfur...,CCNS(=O)(=O)c1ccc(S)c(-c2nc(CC)no2)c1
6438,O=C(/C=C/c1ccccc1)C1(O)CCCCC1,Replace Dialkylaminomethyleneimine with Sulfur...,O=C(/C=C/c1ccccc1)C1(S)CCCCC1
6439,O=C(COc1ccc(Cl)cc1Cl)NCCNC(=O)c1ccc2ccccc2n1,Replace Dialkylaminomethyleneimine with Sulfur...,O=C(COc1ccc(Cl)cc1Cl)SCCNC(=O)c1ccc2ccccc2n1
6440,CC(=O)[C@@H]1CC[C@H]2[C@H]3CC[C@H]4CCC(=O)C[C@...,Replace Dialkylaminomethyleneimine with Sulfur...,CC(=O)[C@@H]1CC[C@H]2[C@H]3CC[C@H]4CCC(S)C[C@]...


In [48]:
prop_name, opt_direction, task_objective, threshold = get_task_info(constraint='strict', task_id=103)
prop_fns = get_prop_function()

smiles_df['reward'] = None
for index, row in tqdm(smiles_df.iterrows()):
    mol = row['mol']
    next_mol = row['next_mol']

    sim = calculate_tanimoto_similarity(mol, next_mol)

    if is_valid_smiles(mol) and is_valid_smiles(next_mol):
        mol_prop = {prop_nm: prop_fns[prop_nm](mol) for prop_nm in prop_name}
        next_mol_prop = {prop_nm: prop_fns[prop_nm](next_mol) for prop_nm in prop_name}
        valid_val = 1
        reward = get_reward(prop_name=prop_name, root_prop=mol_prop, new_prop=next_mol_prop, root_sim=sim, valid_val=valid_val, opt_direction=opt_direction, threshold=threshold)
    else:
        reward = -1

    smiles_df.at[index, 'reward'] = reward

8421it [00:34, 243.17it/s]


In [49]:
smiles_df[smiles_df['reward'] != 0]

,mol,action,next_mol,reward
0,CCCc1cc(N2CCn3c(CC)nnc3C2)n2ncnc2n1,Replace Nitro Group with Carbon Atom.,CCCc1cc(C2CCn3c(CC)nnc3C2)n2ncnc2n1,0.014725
3,CCCc1nnc(NC(=O)CSc2nnnn2-c2ccccc2)s1,Replace Nitro Group with Carbon Atom.,CCCc1ccc(NC(=O)CSc2nnnn2-c2ccccc2)s1,0.010517
4,CCN(CC(=O)NC(C)(C)C)C(=O)c1cnc2c(C)cccn2c1=O,Replace Nitro Group with Carbon Atom.,CCN(CC(=O)NC(C)(C)C)C(=O)c1ccc2c(C)cccn2c1=O,0.014618
9,O=C1C[C@H](c2cc3c(cc2[N+](=O)[O-])OCO3)c2c(c3c...,Replace Nitro Group with Carbon Atom.,O=C1C[C@H](c2cc3c(cc2C)OCO3)c2c(c3ccccc3oc2=O)O1,1.282745
18,C[C@@H]1C[NH+](C[C@@H]2C[C@H](C(C)(C)C)CC[C@@H...,Replace Nitro Group with Carbon Atom.,C[C@@H]1C[NH+](C[C@@H]2C[C@H](C(C)(C)C)CC[C@@H...,0.031888
...,...,...,...,...
8403,COc1ccc(Nc2cc(C)ccc2N)cc1C,Replace Dialkylaminomethyleneimine with Acetoh...,COc1ccc(NC(=O)OCC(=O)O)cc1C,0.026013
8404,O=C(CCC(=O)c1ccc2c(c1)OCCO2)NCCc1cccs1,Replace Dialkylaminomethyleneimine with Acetoh...,O=C(CCC(=O)c1ccc2c(c1)OCCO2)N(C(C)=O)C,0.000622
8407,O=C([O-])[C@@]1([NH2+]C2CC2)CC[C@H](OC2CCC2)C1,Replace Dialkylaminomethyleneimine with Acetoh...,O=C(O)C[C@@H]1CN(C2CC2)C[C@H](OC2CCC2)C1,0.334254
8411,CCOC(=O)c1sc(NC(=O)c2ccc(-c3cccc(Cl)c3)o2)c(C#...,Replace Dialkylaminomethyleneimine with Acetoh...,CCOC(=O)c1sc(NC(=O)C(C)=O)c(C#N)c1C,0.093803


In [50]:
effective_df = smiles_df[smiles_df['reward'] != 0].reset_index()

In [51]:
from collections import Counter

action_list = effective_df.sort_values(by='reward', ascending=False).head(2000)['action'].to_list()
count_dict = Counter(action_list)

In [74]:
count_dict

Counter({'Replace Dialkylaminomethyleneimine with Aromatic Amine.': 41,
         'Replace Guanidine with Carboxylic Acid.': 36,
         'Remove a Guanidine.': 33,
         'Replace Dialkylaminomethyleneimine with Nitrogen Atom.': 32,
         'Replace Ester with Nitrogen Atom.': 32,
         'Replace Dialkylaminomethyleneimine with Amide.': 31,
         'Replace N,N-Dimethylacrylamide with Aromatic Alcohol.': 31,
         'Remove a Nitro Group.': 31,
         'Replace Dialkylaminomethyleneimine with Aliphatic Alcohol.': 30,
         'Replace Guanidine with Aromatic Amine.': 30,
         'Replace Guanidine with Tertiary Amine.': 29,
         'Replace Acetohydroxamate with Oxygen Atom.': 29,
         'Replace Dialkylaminomethyleneimine with Aliphatic Amine.': 28,
         'Replace Guanidine with Aromatic Alcohol.': 28,
         'Replace Guanidine with Amide.': 27,
         'Replace Acetohydroxamate with Aromatic Alcohol.': 26,
         'Replace Nitro Group with Ether.': 26,
         'Re

In [65]:
pd.set_option('display.max_rows', None)
effective_df = effective_df.sort_values(by='reward', ascending=False)
pd.reset_option('display.max_rows')

In [67]:
action_reward_dict = {}
for action in effective_df['action'].to_list():
    action_reward_dict[action] = np.average(effective_df[effective_df['action'] == action]['reward'])

In [68]:
action_reward_dict

{'Replace Dialkylaminomethyleneimine with N,N-Dimethylacrylamide.': 0.663899476991848,
 'Replace Nitro Group with Oxygen Atom.': 0.3446293966141152,
 'Replace Nitro Group with Bromoalkane.': 0.9241200953964649,
 'Replace Dialkylaminomethyleneimine with Aliphatic Alcohol.': 0.5360280949258159,
 'Replace Ester with Aromatic Alcohol.': 0.5118103678704142,
 'Replace Dialkylaminomethyleneimine with Ketone.': 0.47418480042524136,
 'Replace Dialkylaminomethyleneimine with Aromatic Amine.': 0.7966770551162266,
 'Replace Guanidine with Acetylide.': 0.6463685353584547,
 'Replace Guanidine with Tertiary Amine.': 0.6415964556441713,
 'Replace Dialkylaminomethyleneimine with Amide.': 0.2846204396041534,
 'Replace Dialkylaminomethyleneimine with Aliphatic Amine.': 0.4172914963422426,
 'Replace Ketone with Tertiary Amine.': 0.4491590988668097,
 'Replace Guanidine with Aromatic Amine.': 0.4517205821260318,
 'Replace Dialkylaminomethyleneimine with Tertiary Amine.': 0.273862172365508,
 'Replace Guanidi

In [73]:
dict(sorted(action_reward_dict.items(), key=lambda item: item[1], reverse=True))['Replace Nitro Group with Carbon Atom.']

0.29486454171228516

In [61]:
ef_df = effective_df[effective_df.sort_values(by='reward', ascending=False)['action'] == "Replace Nitro Group with Alkene."]
np.average(ef_df['reward'])

C:\Users\25540\AppData\Local\Temp\ipykernel_30004\651539803.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ef_df = effective_df[effective_df.sort_values(by='reward', ascending=False)['action'] == "Replace Nitro Group with Alkene."]


0.3252711116192069

In [9]:
import torch
rl_model_path = "results/rl_model_checkpoint/general_replay_buffer_mol_epi_50_loose_103_best_reward_v5.pth"
if not torch.cuda.is_available():
    rl_model_dict = torch.load(rl_model_path, map_location=torch.device("cpu"))
else:
    rl_model_dict = torch.load(rl_model_path)
state_dim = rl_model_dict['state_dim']
hidden_dim = rl_model_dict['hidden_dim']
action_dim = rl_model_dict['action_dim']
actor_lr = rl_model_dict['actor_lr']
critic_lr = rl_model_dict['critic_lr']
alpha_lr = rl_model_dict['alpha_lr']
target_entropy = rl_model_dict['target_entropy']
tau = rl_model_dict['tau']
gamma = rl_model_dict['gamma']
if torch.cuda.is_available():
    device = rl_model_dict['device']
else:
    device = torch.device("cpu")
rl_planner = RL_Planner(state_dim, hidden_dim, action_dim, actor_lr, critic_lr, alpha_lr, target_entropy, tau, gamma, device)
rl_planner.load_state_dict(rl_model_dict['state_dict'])
action_list = get_all_general_action_list()

In [10]:
from transformers import AutoModel, AutoTokenizer

def str_2_emb(smiles_list, tokenizer, model):

    inputs = tokenizer(smiles_list, return_tensors="pt")

    outputs = model(**inputs)

    embeddings = outputs.last_hidden_state
    smiles_embedding = embeddings[:, 0, :]
    return smiles_embedding

model_name = "seyonec/ChemBERTa-zinc-base-v1"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

d:\Anaconda\anaconda3\envs\dgl\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
'Add a Sulfur Atom. ' in action_list

True

In [33]:
for index, row in effective_df.iterrows():
    mol = row['mol']
    next_mol = row['next_mol']
    state = np.array(str_2_emb([mol], tokenizer=tokenizer, model=model).squeeze().detach())
    mol_action_list = get_general_action_list(mol)
    mask_id = [action_list.index(item) for item in mol_action_list]
    action = rl_planner.get_action(state, mask_id)
    print(action_list[action])
    print(count_dict[action_list[action]])
    if index == 20:
        break

Replace Oxygen Atom with Cyano. 
1
Replace Thioalkyl ether with Acetohydroxamate. 
2
Replace Secondary Amine with Carboxylic Acid. 
2
Remove a Dialkylaminomethyleneimine. 
9
Replace Oxygen Atom with Cyano. 
1
Replace Secondary Amine with Carboxylic Acid. 
2
Replace Oxygen Atom with Cyano. 
1
Replace Secondary Amine with Carboxylic Acid. 
2
Replace Oxygen Atom with Cyano. 
1
Replace Carbon Atom with Secondary Amine. 
1
Remove a Dialkylaminomethyleneimine. 
9
Replace Secondary Amine with Carboxylic Acid. 
2
Replace Carbon Atom with Secondary Amine. 
1
Replace Secondary Amine with Carboxylic Acid. 
2
Replace Oxygen Atom with Cyano. 
1
Replace Oxygen Atom with Cyano. 
1
Replace Oxygen Atom with Cyano. 
1
Replace Secondary Amine with Carboxylic Acid. 
2
Replace Oxygen Atom with Cyano. 
1
Replace Ester with Secondary Amine. 
6
Replace Oxygen Atom with Cyano. 
1


In [39]:
top_100 = count_dict.most_common(100)

action_list = []
for item, count in top_100:
    action_list.append(item)
    # print(f"{item}: {count}")


In [45]:
count_dict

Counter({'Replace Nitro Group with Carbon Atom. ': 31,
         'Replace Nitro Group with Alkene. ': 30,
         'Replace Nitro Group with Chlorine Atom. ': 28,
         'Replace Nitro Group with Cyano. ': 26,
         'Replace Nitro Group with Fluoroalkane. ': 26,
         'Replace Nitro Group with Fluorine Atom. ': 25,
         'Remove a Nitro Group. ': 23,
         'Replace Guanidine with Alkene. ': 20,
         'Replace Nitro Group with Tertiary Amine. ': 20,
         'Replace Nitro Group with Secondary Amine. ': 20,
         'Replace Nitro Group with Bromine Atom. ': 19,
         'Remove a Guanidine. ': 19,
         'Replace Nitro Group with Carboxylic Acid. ': 19,
         'Replace Nitro Group with Chloroalkane. ': 19,
         'Replace Nitro Group with Bromoalkane. ': 18,
         'Replace Nitro Group with Aliphatic Alcohol. ': 16,
         'Replace Dialkylaminomethyleneimine with Carboxylic Acid. ': 15,
         'Replace Nitro Group with Aliphatic Amine. ': 15,
         'Repla

In [42]:
with open('action_list.txt', 'w', encoding='utf-8') as file:
    for item in action_list:
        # 写入元素并添加换行符
        file.write(item + '\n')

In [43]:
read_list = []
with open('action_list.txt', 'r', encoding='utf-8') as file:
    # 读取文件的所有行，返回一个包含每行内容的列表
    lines = file.readlines()
    # 遍历每行内容
    for line in lines:
        # 去除每行末尾的换行符，并添加到 my_list 中
        read_list.append(line.strip())

# 打印转换后的列表
print(read_list == action_list)

False
